In [1]:
# -*- coding: utf-8 -*-
import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import pandas as pd
from pathlib import Path
import os
import glob

In [7]:
# !wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'
# !unzip stanford-ner-2018-10-16.zip

In [9]:
st = StanfordNERTagger('filepath/stanford-ner-2018-10-16/classifiers/english.muc.7class.distsim.crf.ser.gz',
                       'filepath/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

In [3]:
def page_ner(filepath:str,vol_id):
  page_text= open(filepath,errors='replace').read()
  tokenized_text = word_tokenize(page_text)
  classified_text = st.tag(tokenized_text)
  word_lst=[]
  #class_lst=[]
  for item in classified_text:
    #print(type(item))#<class 'tuple'>
    aList = list(item)
    #word=aList[0]
    #cls=aList[-1]
    #word_lst.append(word)
    #class_lst.append(cls)
    word_lst.append(aList)
  #ner_df = pd.DataFrame({'word': word_lst,'ner': class_lst})
  ner_df = pd.DataFrame(word_lst,columns=["word","ner"])
  #select_ner_df = ner_df.loc[ner_df['ner'] != 'O']
  #select_ner_df = select_ner_df.drop_duplicates() #deduplicate
  page_id= filepath.split('/')[-1].rstrip('.txt')
  ner_df['vol_id'] = vol_id
  ner_df['page_id'] = page_id

  temp_ent_type = None
  prev_idx = -1
  entity_list = []
  start = -1
  for x in ner_df[ner_df["ner"]!="O"].to_records():
    if (temp_ent_type!=x["ner"]) or (x.index-1!=prev_idx):
      if temp_ent_type is not None:
        entity_list.append((" ".join(temp_ent),temp_ent_type,x.vol_id,x.page_id,start,prev_idx+1))
      temp_ent_type = x["ner"]
      temp_ent = [x["word"]]
      start = x.index
    else:
      temp_ent = temp_ent + [x["word"]]
    prev_idx = x.index

  # last entity
  if temp_ent_type is not None:
    entity_list.append((" ".join(temp_ent),temp_ent_type,x.vol_id,x.page_id,start,prev_idx+1))
  
  ner_df = pd.DataFrame(entity_list,columns = ["word","ner","vol_id","page_id","start_token","end_token"])
  return ner_df

In [4]:
def process_workset(workset_path:str):
   df_lst=[]
   vol_paths = Path(workset_path).glob('*')
   for vol_path in vol_paths:
      print(vol_path)
      # print(type(vol_path)) # <class 'pathlib.PosixPath'>
      vol_path_str=vol_path.as_posix()#convert to str
      vol_id=vol_path_str.split('/')[-1]
      print(vol_id)
      os.chdir(vol_path)
      extension = 'txt'
      all_pages = [i for i in glob.glob('*.{}'.format(extension))]
      for page in all_pages:
        filepath=vol_path_str+'/'+page #fullpath
        #print(filepath)
        sub_df=page_ner(filepath,vol_id)
        df_lst.append(sub_df)
        #print(len(df_lst))
   df_combined=pd.concat(df_lst) 
   return df_combined


In [10]:
test_df = process_workset('filepath/testing_volume/')

test_df.to_csv('filepath/outcomes.csv')